In [1]:
import jax.numpy as jnp
import numpy as np

In [2]:
x = jnp.arange(25, dtype=jnp.float32).reshape(5, 5)
print(x)

[[ 0.  1.  2.  3.  4.]
 [ 5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14.]
 [15. 16. 17. 18. 19.]
 [20. 21. 22. 23. 24.]]


In [21]:
y = np.arange(25, dtype=float).reshape(5, 5)
print(y)

[[ 0.  1.  2.  3.  4.]
 [ 5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14.]
 [15. 16. 17. 18. 19.]
 [20. 21. 22. 23. 24.]]


In [22]:
x.T

DeviceArray([[ 0.,  5., 10., 15., 20.],
             [ 1.,  6., 11., 16., 21.],
             [ 2.,  7., 12., 17., 22.],
             [ 3.,  8., 13., 18., 23.],
             [ 4.,  9., 14., 19., 24.]], dtype=float32)

In [25]:
x_gram = jnp.dot(x, x.T).block_until_ready()
print(x_gram)

[[  30.   80.  130.  180.  230.]
 [  80.  255.  430.  605.  780.]
 [ 130.  430.  730. 1030. 1330.]
 [ 180.  605. 1030. 1455. 1880.]
 [ 230.  780. 1330. 1880. 2430.]]


In [17]:
y_gram = np.dot(y, y.T)
print(y_gram)

[[  30.   80.  130.  180.  230.]
 [  80.  255.  430.  605.  780.]
 [ 130.  430.  730. 1030. 1330.]
 [ 180.  605. 1030. 1455. 1880.]
 [ 230.  780. 1330. 1880. 2430.]]


In [26]:
from jax import jit

@jit
def static_jax_dot():
    x = jnp.arange(25, dtype=jnp.float32).reshape(5, 5)
    x_gram = jnp.dot(x, x.T)
    return x_gram

static_jax_dot().block_until_ready()

DeviceArray([[  30.,   80.,  130.,  180.,  230.],
             [  80.,  255.,  430.,  605.,  780.],
             [ 130.,  430.,  730., 1030., 1330.],
             [ 180.,  605., 1030., 1455., 1880.],
             [ 230.,  780., 1330., 1880., 2430.]], dtype=float32)

In [31]:
from functools import partial

@partial(jit,static_argnames=['size'])
def static_jax_dot(size):
    x = jnp.arange(size**2, dtype=jnp.float32).reshape(5, 5)
    x_gram = jnp.dot(x, x.T)
    return x_gram

static_jax_dot(5).block_until_ready()

DeviceArray([[  30.,   80.,  130.,  180.,  230.],
             [  80.,  255.,  430.,  605.,  780.],
             [ 130.,  430.,  730., 1030., 1330.],
             [ 180.,  605., 1030., 1455., 1880.],
             [ 230.,  780., 1330., 1880., 2430.]], dtype=float32)

In [2]:
import timeit
import numpy as np
import jax.numpy as jnp
from jax import jit
from functools import partial
import pandas as pd

# (size, size)の行列を作ってMod計算
@partial(jit, static_argnames=['size'])
def jax_jit_mod(size):
    x = jnp.arange(size, dtype=jnp.int32)
    mat = x[None, :] * x[:, None] # (size, size)
    return mat

def jax_nojit_mod(size):
    x = jnp.arange(size, dtype=jnp.int32)
    mat = x[None, :] * x[:, None]
    return mat

def numpy_mod(size):
    x = np.arange(size, dtype=np.int32)
    mat = x[None, :] * x[:, None]
    return mat

numpy = []
jax_nojit = []
jax_jit = []
indices = []
sizes = [10, 100, 1000, 10000, 30000]
for i in sizes:
    size = i
    repeat = 1000
    
    indices.append(size)
    numpy.append(timeit.timeit(lambda: numpy_mod(size), number=repeat))
    jax_nojit.append(timeit.timeit(lambda: jax_nojit_mod(size), number=repeat))
    jax_jit.append(timeit.timeit(lambda: jax_jit_mod(size), number=repeat))
#--

df = pd.DataFrame({'numpy': numpy, 'jax_nojit': jax_nojit, 'jax_jit':jax_jit}, index=indices)
df

numpy   jax_nojit     jax_jit
10       0.002824    0.200248    0.014847
100      0.003904    0.163214    0.013364
1000     0.337542    0.357501    0.190362
10000   38.935593   19.819444   20.075136
30000  364.126992  180.609012  182.957503